In [2]:
from flask import Flask, render_template, request
import re
import random
import json

In [3]:
def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s
# remove spacial and residual character 
def special_characters(str):
    str = re.sub(r"[^a-eg-ik-vxyA-EG-IK-VXY0-9 ]","",str)
    index = 0
    for i in str:
        if index != 0:
            if 'sxr'.find(i)>=0:
                if str[index-1] != ' ':
                    str =str.replace(i,"")
                    index-=1
        index+=1
    return str
# find a like b
def like(a,b):
    a = a.split(" ")
    for i in a:
        if b.find(i) < 0:
            return False
    return True

In [4]:
app = Flask(__name__)
@app.route("/")
def home():
    return render_template("index.html")
# get reply 
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    db = request.args.get('DB')
    if db == "":
        with open("data/InforGroup.json",encoding='utf-8') as file:
            data = json.load(file)
    else:
        with open("data/"+db+".json",encoding='utf-8') as file:
            data = json.load(file)
    for intents in data["intents"]:
        for tag in intents["tag"]:
            if like(special_characters(no_accent_vietnamese(str(tag))),special_characters(no_accent_vietnamese(str(userText)))):
                for questions in intents['questions']:
                    if special_characters(no_accent_vietnamese(userText)).find(special_characters(no_accent_vietnamese(questions))) >=0:
                        return intents["answers"][random.randint(0, len(intents["answers"])-1)]
    for intents in data["intents"]:
        for questions in intents['questions']:
            if special_characters(no_accent_vietnamese(userText)).find(special_characters(no_accent_vietnamese(questions))) >=0:
                return intents["answers"][random.randint(0, len(intents["answers"])-1)]
    return "Thông tin này AI chưa được học :D"       

# run main
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2022 16:21:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 16:21:08] "GET /static/style.css HTTP/1.1" 304 -
